In [3]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from pprint import pprint
from inspect import getmembers
from gurobipy import *
import numpy as np
import random as random
import math
import copy
import timeit
import plotly.express as px
import plotly.graph_objs as go

'''
recursive function that returns the binary path: 0 for left, 1 for right to all leaves in a dictionary
-input: treenode= starting node (0)
        binary_path= initialised with -1's
-output: a dictionary with a key for each leaf, and an array (length=num_nodes in tree) with 0 for left, 1 for right at each split, -1 if split doesn't arrive at leaf
'''
def get_binary_path_to_leaf(binary_path,tree,node):

    # go to next node if not leaf
    if tree.children_left[node]>0:
        binary_path_left=np.copy(binary_path)
        binary_path_left[node]=0
        binary_path_right=np.copy(binary_path)
        binary_path_right[node]=1
        child_list_left=get_binary_path_to_leaf(binary_path_left,tree,tree.children_left[node])
        child_list_right=get_binary_path_to_leaf(binary_path_right,tree,tree.children_right[node])
    else:
        # pass back dictionary
        return {node:binary_path}

    #combine dictionaries
    dall = {}
    dall.update(child_list_left)
    dall.update(child_list_right)
    #print("dall",dall)
    return dall


'''
Returns the depth of each node in the tree
-input: tree object
-output: (n_node array) an array with the depth of each node at the index of the node
'''
def get_depth(tree):

    num_nodes=tree.node_count
    depth_node=np.zeros([num_nodes])
    current_nodes=[0]
    current_depth=0
    new_nodes=[]
    while current_nodes:
        for node in current_nodes:

            if not(tree.children_left[node] == -1):
                new_nodes.append(tree.children_left[node])

            if not(tree.children_right[node] == -1):
                new_nodes.append(tree.children_right[node])

            depth_node[node]=current_depth

        current_nodes=new_nodes
        new_nodes=[]
        current_depth+=1

    return depth_node

'''
truncate tree to a certain depth
'''

def truncate_to_given_depth(forest,max_depth):

    estimators=forest.estimators_

    for tree_index,tree_in_forest in enumerate(estimators):

        tree=tree_in_forest.tree_
        depth=get_depth(tree)
        num_nodes=tree.node_count

        for node in range(num_nodes):
            if depth[node]>=max_depth:
                tree.children_left[node]=-1
                tree.children_right[node]=-1

    return forest


'''
Returns a a list of indices corresponding to the leaves of a tree
'''
def get_leaf_indices(tree):

    current_nodes=[0]
    leaf_indicies=[]
    new_nodes=[]
    while current_nodes:
        for node in current_nodes:

            if not(tree.children_left[node] == -1):
                new_nodes.append(tree.children_left[node])
            else: leaf_indicies.append(node)

            if not(tree.children_right[node] == -1):
                new_nodes.append(tree.children_right[node])

        current_nodes=new_nodes
        new_nodes=[]

    return leaf_indicies

'''
Input - sklearn random forest
Output - splits: (num_featres x num_splits per feature) ordered list with threshold defining splits in all trees
'''
def get_ordered_thresholds_for_each_feature(forest, num_features):
    splits = []
    for feat in range(num_features):
        splits.append([])

    for tree_index,tree_in_forest in enumerate(forest):
        tree=tree_in_forest.tree_
        index_splits=np.where(tree.children_left!= (-1))[0]

        for i in index_splits:
            var_split=tree.feature[i]
            splits[var_split].append(tree.threshold[i])

    for feat in range(num_features):
        splits[feat].sort()

    return splits




In [6]:
def choose_dataset(dataset,n_data,n_feature):

    if dataset == 'wine':
        # n_trees_train=10
        x = pd.read_csv('winequality.csv', sep=',')
        wine_headings = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
                         'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']
        y = x[["quality"]]
        del x['quality']

        feature_upper_bounds = [15.9, 1.58, 1, 15.5, 0.611, 72, 289, 1.00369, 4.01, 2, 14.9]
        feature_lower_bounds = [4.6, 0.12, 0, 0.9, 0.012, 1, 6, 0.99007, 2.74, 0.33, 8.4]

        n_feature=len(wine_headings)
        n_data=len(y)
        

    if dataset == 'concrete':
        # n_trees_train=10
        x = pd.read_csv('concrete.csv', sep=',')
        y = x[["CompressiveStrength"]]
        del x['CompressiveStrength']

        feature_upper_bounds = [540,359.4,200.1,247,32.2,1145,992.6,365,82.6]
        feature_lower_bounds = [102,0,0,121.8,0,801,594,1,2.33]

        n_feature=x.shape[1]
        n_data=x.shape[0]
        
    if dataset == 'linear_synth':

        n_data = 5000
        n_feature = 10

        feature_upper_bounds=[5]*n_feature
        feature_lower_bounds=[-5]*n_feature

        x=np.random.randint(10,  size=(n_data,n_feature))-5

        # a simple linear model (still hard to learn for a tree)
        beta=np.random.rand(n_feature)
        y=x.dot(beta)
        
    if dataset == 'abs_synth':

        feature_upper_bounds=[2]*n_feature
        feature_lower_bounds=[0]*n_feature

        x=np.random.rand(n_data,n_feature)*2
        y=np.sum(1-np.abs(x-1),1)
        
    if dataset == 'abs_synth_noisy':

        feature_upper_bounds=[2]*n_feature
        feature_lower_bounds=[0]*n_feature

        x=np.random.rand(n_data,n_feature)*2
        y=np.sum(1-np.abs(x-1),1) + np.random.rand(n_data)*n_feature
#         y=np.sum(1-np.abs(x-1),1) + np.random.normal(0,1,n_data)

        
        
    return x,y,feature_upper_bounds,feature_lower_bounds,n_data,n_feature

In [14]:
def get_leaf_indices_recursive(tree, curr_index):
    if tree.children_left[curr_index] > -1:
        leaves_a=get_leaf_indices_recursive(tree,tree.children_left[curr_index])
        leaves_b=get_leaf_indices_recursive(tree,tree.children_right[curr_index])
        return leaves_a | leaves_b 
    else:
        return {curr_index}

def add_deviation_bound(m,feature_var,x,num_total_features):

    mean_x=x.mean()
    sd_x=np.sqrt(x.var())
    norm_dev={}
    for feat in range(num_total_features):
        norm_dev[feat]=m.addVar(lb=-GRB.INFINITY,name="norm_dev"+str(feat))
        if  type(x.mean())==pd.core.series.Series:
            m.addConstr(norm_dev[feat] >= (feature_var[feat]- mean_x[feat])/ sd_x[feat])
            m.addConstr(norm_dev[feat] >= (-feature_var[feat] + mean_x[feat])/ sd_x[feat])
        else:
            m.addConstr(norm_dev[feat] >= (feature_var[feat]- mean_x)/ sd_x)
            m.addConstr(norm_dev[feat] >= (-feature_var[feat] + mean_x)/ sd_x)

    m.addConstr(quicksum(norm_dev[feat] for feat in range(num_total_features)) <= 0.1*num_total_features)
                   
    return m


In [1]:
 # misic formulation
def misic_opt(estimators,num_total_features,feature_upper_bounds,feature_lower_bounds,x,eps,make_continuous,deviation_bound,make_relax):

    m = Model("RF")
    m.setParam('TimeLimit', 1800)
#     m.setParam('TimeLimit', 60)

    num_trees=len(estimators)
    in_leaf_var={}
    split_var={}

    for tree_index,tree_in_forest in enumerate(estimators):
        tree=tree_in_forest.tree_
        index_leaves=np.array(get_leaf_indices(tree))

        for leaf in index_leaves:
            in_leaf_var[leaf,tree_index]=m.addVar(vtype=GRB.BINARY,name="in_leaf_var"+str(leaf)+"tree_index"+str(tree_index))

        m.addConstr(quicksum(in_leaf_var[leaf,tree_index] for leaf in index_leaves)==1)

    for feat in range(n_feature):
        feat_indices_all=[]
        tree_indices_all=[]
        feat_thresholds_all=[]

        for tree_index,tree_in_forest in enumerate(estimators):

            tree=tree_in_forest.tree_
            feat_indices=np.where(tree.feature==feat)[0]
            feat_thresholds=tree.threshold[tree.feature==feat]
            order_indices=np.argsort(feat_thresholds)

            leaf_indices=set()

            for i in range(len(feat_indices)):
                split_var[feat,feat_indices[order_indices][i],tree_index]=m.addVar(vtype=GRB.BINARY,name="split_var"+str(feat)+"-"+str(feat_indices[order_indices][i])+"_tree"+str(tree_index))

                new_leaf_indices=get_leaf_indices_recursive(tree,tree.children_left[feat_indices[order_indices][i]])
                right_leaf_indices=get_leaf_indices_recursive(tree,tree.children_right[feat_indices[order_indices][i]])


                leaf_indices= new_leaf_indices | leaf_indices
                m.addConstr(split_var[feat,feat_indices[order_indices][i],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in new_leaf_indices))
                m.addConstr(1 - split_var[feat,feat_indices[order_indices][i],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in right_leaf_indices))

            feat_indices_all=np.append(feat_indices_all,feat_indices)
            feat_thresholds_all=np.append(feat_thresholds_all,feat_thresholds)
            tree_indices_all=np.append(tree_indices_all,np.ones(len(feat_indices))*tree_index)

        indices_to_sort=np.argsort(feat_thresholds_all)
        ordered_thresholds=feat_thresholds_all[indices_to_sort]
        ordered_tree_indices=tree_indices_all[indices_to_sort]
        ordered_indices=feat_indices_all[indices_to_sort]

        for i in range(len(indices_to_sort)-1):
            m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       <= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])
            if ordered_thresholds[i]==ordered_thresholds[i+1]:
                m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       >= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])


    # Summing objective over all trees
    m.setObjective(quicksum(quicksum(in_leaf_var[leaf,tree_index]*tree_in_forest.tree_.value[leaf,0,0] 
                                     for leaf in np.array(get_leaf_indices(tree_in_forest.tree_))) for tree_index,tree_in_forest in enumerate(estimators))/float(num_trees),GRB.MAXIMIZE)

    m.write("new_model_file.lp")
    
    leaf_out=np.ones([num_trees])
    if make_relax==True:
        relax = m.relax()
        relax.optimize()
        obj = relax.getObjective()
    else:
        m.optimize()
        obj = m.getObjective()
        for tree_index,tree_in_forest in enumerate(estimators):
            tree=tree_in_forest.tree_
            index_leaves=np.array(get_leaf_indices(tree))
            for i in index_leaves:
                if m.getVarByName("in_leaf_var"+str(i)+"tree_index"+str(tree_index)).x==1:
                    leaf_out[tree_index]=i

    return obj.getValue(),leaf_out, m.Runtime



# expset formulation
def misic_opt_tighter(estimators,num_total_features,feature_upper_bounds,feature_lower_bounds,x,eps,make_continuous,deviation_bound,make_relax):

    m = Model("RF both ways greater and less than")
    m.setParam('TimeLimit', 1800)
#     m.setParam('TimeLimit', 60)
    num_trees=len(estimators)
    in_leaf_var={}
    split_var={}

    for tree_index,tree_in_forest in enumerate(estimators):
        tree=tree_in_forest.tree_
        index_leaves=np.array(get_leaf_indices(tree))

        for leaf in index_leaves:
            in_leaf_var[leaf,tree_index]=m.addVar(vtype=GRB.BINARY,name="in_leaf_var"+str(leaf)+"tree_index"+str(tree_index))

        m.addConstr(quicksum(in_leaf_var[leaf,tree_index] for leaf in index_leaves)==1)

    for feat in range(n_feature):

        print('feat',feat)

        feat_indices_all=[]
        tree_indices_all=[]
        feat_thresholds_all=[]

        for tree_index,tree_in_forest in enumerate(estimators):

            print('tree_index',tree_index)

            tree=tree_in_forest.tree_
            feat_indices=np.where(tree.feature==feat)[0]
            feat_thresholds=tree.threshold[tree.feature==feat]
            order_indices=np.argsort(feat_thresholds)

            leaf_indices=set()
            n_ind=len(feat_indices)

            for i in range(n_ind):
                split_var[feat,feat_indices[order_indices][i],tree_index]=m.addVar(vtype=GRB.BINARY,name="split_var"+str(feat)+"-"+str(feat_indices[order_indices][i])+"_tree"+str(tree_index))
                new_leaf_indices=get_leaf_indices_recursive(tree,tree.children_left[feat_indices[order_indices][i]])
                leaf_indices= new_leaf_indices | leaf_indices
                m.addConstr(split_var[feat,feat_indices[order_indices][i],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in leaf_indices))

            larger_leaf_indices=set()

            for i in range(n_ind):
                right_leaf_indices=get_leaf_indices_recursive(tree,tree.children_right[feat_indices[order_indices][n_ind-i-1]])
                larger_leaf_indices= right_leaf_indices | larger_leaf_indices
                m.addConstr(1 - split_var[feat,feat_indices[order_indices][n_ind-i-1],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in larger_leaf_indices))

            feat_indices_all=np.append(feat_indices_all,feat_indices)
            feat_thresholds_all=np.append(feat_thresholds_all,feat_thresholds)
            tree_indices_all=np.append(tree_indices_all,np.ones(n_ind)*tree_index)

        indices_to_sort=np.argsort(feat_thresholds_all)
        ordered_thresholds=feat_thresholds_all[indices_to_sort]
        ordered_tree_indices=tree_indices_all[indices_to_sort]
        ordered_indices=feat_indices_all[indices_to_sort]


        for i in range(len(indices_to_sort)-1):
            m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       <= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])
            if ordered_thresholds[i]==ordered_thresholds[i+1]:
                m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       >= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])

    # Summing objective over all trees
    m.setObjective(quicksum(quicksum(in_leaf_var[leaf,tree_index]*tree_in_forest.tree_.value[leaf,0,0] 
                                     for leaf in np.array(get_leaf_indices(tree_in_forest.tree_))) for tree_index,tree_in_forest in enumerate(estimators))/float(num_trees),GRB.MAXIMIZE)

    m.write("new_model_file.lp")
    leaf_out=np.ones([num_trees])
    if make_relax==True:
        relax = m.relax()
        relax.optimize()
        obj = relax.getObjective()
    else:
        m.optimize()
        obj = m.getObjective()
        leaf_out=np.ones([num_trees])
        for tree_index,tree_in_forest in enumerate(estimators):
            tree=tree_in_forest.tree_
            index_leaves=np.array(get_leaf_indices(tree))
            for i in index_leaves:
                if m.getVarByName("in_leaf_var"+str(i)+"tree_index"+str(tree_index)).x==1:
                    leaf_out[tree_index]=i
                    

        print('leaf_out',leaf_out)
    
    return obj.getValue(),leaf_out

# bigM formulation
def tighter_random_forest_IP_unconstrained(estimators,num_total_features,make_relax):

    m = Model("RF")
    m.setParam('TimeLimit', 1800)
    num_trees=len(estimators)

    print("num_total_features",num_total_features)

    #initialize variables
    in_leaf_var={}
    split_var={}
    feature_var={}
    in_jury={}
    tree_obj={}

    for i in range(num_total_features):
        feature_var[i]=m.addVar(name="feature_var"+str(i))

    for tree_index,tree_in_forest in enumerate(estimators):

        tree=tree_in_forest.tree_
        num_nodes=tree.node_count
        ## leaf nodes
        index_leaves=np.array(get_leaf_indices(tree))
        ## interior nodes
        index_splits=np.where(tree.children_left!= (-1))[0]

        # coming up with tight big M values
        max_threshold=max(abs(tree.threshold))+1

        # q^k_{i,r_i} and q^k_{i,l_i} in the model
        for i in index_splits:
            split_var[i,tree.children_left[i],tree_index]=m.addVar(vtype=GRB.BINARY,name="split_var"+str(i)+str(tree.children_left[i])+"tree_index"+str(tree_index))
            split_var[i,tree.children_right[i],tree_index]=m.addVar(vtype=GRB.BINARY,name="split_var"+str(i)+str(tree.children_right[i])+"tree_index"+str(tree_index))
        # z^k_j
        for i in index_leaves:
            in_leaf_var[i,tree_index]=m.addVar(vtype=GRB.BINARY,name="in_leaf_var"+str(i)+"tree_index"+str(tree_index))

        tree_obj[tree_index]=m.addVar(name='tree_obj'+str(tree_index))

        m.update()
        ## adding objective sum_{j leaf} z^k_j S^k_j for a tree k
        m.addConstr(tree_obj[tree_index]==quicksum(in_leaf_var[leaf,tree_index]*tree.value[leaf,0,0] for leaf in index_leaves))

        # branching constraint
        for split in index_splits:
            #Big M constraints
            m.addConstr(feature_var[tree.feature[split]]-max_threshold*(1-split_var[split,tree.children_left[split],tree_index]) <= tree.threshold[split] )
            m.addConstr(feature_var[tree.feature[split]]+max_threshold*(1-split_var[split,tree.children_right[split],tree_index]) >= tree.threshold[split] )

            left=tree.children_left[split]
            if tree.children_left[split] in index_splits:
                left_right=tree.children_right[left]
                left_left=tree.children_left[left]
                # We do not need the first two contraints
                # equal flow in the interior node i
                m.addConstr(split_var[split,left,tree_index] >= split_var[left,left_right,tree_index])
                m.addConstr(split_var[split,left,tree_index] >= split_var[left,left_left,tree_index])
                m.addConstr(split_var[split,left,tree_index] == split_var[left,left_right,tree_index]+split_var[left,left_left,tree_index])
            else:
                m.addConstr(split_var[split,left,tree_index] >= in_leaf_var[left,tree_index])

            right=tree.children_right[split]
            if tree.children_right[split] in index_splits:
                right_right=tree.children_right[right]
                right_left=tree.children_left[right]
                m.addConstr(split_var[split,right,tree_index] == split_var[right,right_left,tree_index]+split_var[right,right_right,tree_index])
                m.addConstr(split_var[split,right,tree_index] >= split_var[right,right_left,tree_index])
                m.addConstr(split_var[split,right,tree_index] >= split_var[right,right_left,tree_index])
            else:
                m.addConstr(split_var[split,right,tree_index] >= in_leaf_var[right,tree_index])
        # sum over z^k_j =1 for tree k
        m.addConstr(quicksum(in_leaf_var[leaf,tree_index] for leaf in index_leaves)==1)
    # Summing objective over all trees
    m.setObjective(quicksum(tree_obj[tree_index] for tree_index,tree_in_forest in enumerate(estimators))/float(num_trees),GRB.MAXIMIZE)
    m.write("unconst_model_file.lp")
    
    feature_var_out=np.ones([num_total_features])
    if make_relax==True:
        relax = m.relax()
        relax.optimize()
        obj = relax.getObjective()
    else:
        m.optimize()
        '''
        #m.write("model_file.lp")
        m.computeIIS()
        m.write("model.ilp")
        print("IIS written to file 'model.ilp'")
        print("STATUS", m.Status)
        '''

        '''
        # output the results
        for tree_index,tree_in_forest in enumerate(estimators):

            tree=tree_in_forest.tree_
            index_leaves=np.where(tree.children_left== (-1))[0]
            index_splits=np.where(tree.children_left!= (-1))[0]

            #in_leaf=np.empty([num_leaves])
            #print("index_leaves",index_leaves)
            for leaf in index_leaves:
                print(m.getVarByName("in_leaf_var"+str(leaf)+"tree_index"+str(tree_index)),
                      m.getVarByName("in_leaf_var"+str(leaf)+"tree_index"+str(tree_index)).x)
                #in_leaf[leaf]=m.getVarByName("in_leaf_var"+str(leaf)).x

            #splits=np.empty([num_splits])
            for split in index_splits:
                print(m.getVarByName("split_var"+str(split)+"tree_index"+str(tree_index)),
                      m.getVarByName("split_var"+str(split)+"tree_index"+str(tree_index)).x)
                #splits[split]=m.getVarByName("split_var"+str(split)).x
        '''

        for i in range(num_total_features):
            feature_var_out[i]=m.getVarByName("feature_var"+str(i)).x

        print("feature_var_out",feature_var_out)
        obj = m.getObjective()
        print("tight_formulation obj",obj.getValue())


    return feature_var_out,obj.getValue(), m.Runtime



# elbow+expset formulation
def misic_opt_tighter_with_cuts(estimators,num_total_features,feature_upper_bounds,feature_lower_bounds,x,eps,make_continuous,deviation_bound,make_relax):

    m = Model("RF both ways greater and less than")
#     m.setParam('TimeLimit', 60)
    m.setParam('TimeLimit', 1800)
    num_trees=len(estimators)
    in_leaf_var={}
    split_var={}

    for tree_index,tree_in_forest in enumerate(estimators):
        tree=tree_in_forest.tree_
        index_leaves=np.array(get_leaf_indices(tree))

        for leaf in index_leaves:
            in_leaf_var[leaf,tree_index]=m.addVar(vtype=GRB.BINARY,name="in_leaf_var"+str(leaf)+"tree_index"+str(tree_index))

        m.addConstr(quicksum(in_leaf_var[leaf,tree_index] for leaf in index_leaves)==1)


    for feat in range(n_feature):

        print('feat',feat)

        feat_indices_all=[]
        tree_indices_all=[]
        feat_thresholds_all=[]

        for tree_index,tree_in_forest in enumerate(estimators):

            print('tree_index',tree_index)

            tree=tree_in_forest.tree_
            feat_indices=np.where(tree.feature==feat)[0]
            feat_thresholds=tree.threshold[tree.feature==feat]
            order_indices=np.argsort(feat_thresholds)

            leaf_indices=set()
            n_ind=len(feat_indices)

            for i in range(n_ind):
                split_var[feat,feat_indices[order_indices][i],tree_index]=m.addVar(vtype=GRB.BINARY,name="split_var"+str(feat)+"-"+str(feat_indices[order_indices][i])+"_tree"+str(tree_index))

                new_leaf_indices=get_leaf_indices_recursive(tree,tree.children_left[feat_indices[order_indices][i]])
                leaf_indices= new_leaf_indices | leaf_indices
                m.addConstr(split_var[feat,feat_indices[order_indices][i],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in leaf_indices))

                # find parent, check if same feature, then add
                current_index=feat_indices[order_indices][i]
                parent_index= np.where(tree.children_right == current_index)[0]
                if tree.feature[parent_index]== feat:
                    m.addConstr(split_var[feat,current_index,tree_index]-split_var[feat,parent_index[0],tree_index] >= 
                                quicksum(in_leaf_var[leaf,tree_index] for leaf in new_leaf_indices))

            larger_leaf_indices=set()

            for i in range(n_ind):
                right_leaf_indices=get_leaf_indices_recursive(tree,tree.children_right[feat_indices[order_indices][n_ind-i-1]])
                larger_leaf_indices= right_leaf_indices | larger_leaf_indices
                m.addConstr(1 - split_var[feat,feat_indices[order_indices][n_ind-i-1],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in larger_leaf_indices))

                #find parent, check if same feature, then add
                current_index=feat_indices[order_indices][n_ind-i-1]
                parent_index= np.where(tree.children_left == current_index)[0]
                if tree.feature[parent_index]== feat:
                    m.addConstr(split_var[feat,parent_index[0],tree_index]-split_var[feat,current_index,tree_index] >= 
                                quicksum(in_leaf_var[leaf,tree_index] for leaf in right_leaf_indices))

            feat_indices_all=np.append(feat_indices_all,feat_indices)
            feat_thresholds_all=np.append(feat_thresholds_all,feat_thresholds)
            tree_indices_all=np.append(tree_indices_all,np.ones(n_ind)*tree_index)

        indices_to_sort=np.argsort(feat_thresholds_all)
        ordered_thresholds=feat_thresholds_all[indices_to_sort]
        ordered_tree_indices=tree_indices_all[indices_to_sort]
        ordered_indices=feat_indices_all[indices_to_sort]

        tree.children_left

        for i in range(len(indices_to_sort)-1):
            m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       <= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])
            if ordered_thresholds[i]==ordered_thresholds[i+1]:
                m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       >= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])

    # Summing objective over all trees
    m.setObjective(quicksum(quicksum(in_leaf_var[leaf,tree_index]*tree_in_forest.tree_.value[leaf,0,0] 
                                     for leaf in np.array(get_leaf_indices(tree_in_forest.tree_))) for tree_index,tree_in_forest in enumerate(estimators))/float(num_trees),GRB.MAXIMIZE)

    m.write("new_model_file.lp")
    leaf_out=np.ones([num_trees])
    if make_relax==True:
        relax = m.relax()
        relax.optimize()
        obj = relax.getObjective()
    else:
        m.optimize()
        obj = m.getObjective()
        leaf_out=np.ones([num_trees])
        for tree_index,tree_in_forest in enumerate(estimators):
            tree=tree_in_forest.tree_
            index_leaves=np.array(get_leaf_indices(tree))
            for i in index_leaves:
                if m.getVarByName("in_leaf_var"+str(i)+"tree_index"+str(tree_index)).x==1:
                    leaf_out[tree_index]=i
    
    return obj.getValue(),leaf_out


# elbow formulation
def misic_opt_tighter_with_cuts_2(estimators,num_total_features,feature_upper_bounds,feature_lower_bounds,x,eps,make_continuous,deviation_bound,make_relax):

    m = Model("RF both ways greater and less than")
#     m.setParam('TimeLimit', 1800)
    m.setParam('TimeLimit', 60)
    num_trees=len(estimators)
    in_leaf_var={}
    split_var={}

    for tree_index,tree_in_forest in enumerate(estimators):
        tree=tree_in_forest.tree_
        index_leaves=np.array(get_leaf_indices(tree))

        for leaf in index_leaves:
            in_leaf_var[leaf,tree_index]=m.addVar(vtype=GRB.BINARY,name="in_leaf_var"+str(leaf)+"tree_index"+str(tree_index))

        m.addConstr(quicksum(in_leaf_var[leaf,tree_index] for leaf in index_leaves)==1)


    for feat in range(n_feature):

        print('feat',feat)

        feat_indices_all=[]
        tree_indices_all=[]
        feat_thresholds_all=[]

        for tree_index,tree_in_forest in enumerate(estimators):

            print('tree_index',tree_index)

            tree=tree_in_forest.tree_
            feat_indices=np.where(tree.feature==feat)[0]
            feat_thresholds=tree.threshold[tree.feature==feat]
            order_indices=np.argsort(feat_thresholds)

            leaf_indices=set()
            n_ind=len(feat_indices)

            for i in range(n_ind):
                split_var[feat,feat_indices[order_indices][i],tree_index]=m.addVar(vtype=GRB.BINARY,name="split_var"+str(feat)+"-"+str(feat_indices[order_indices][i])+"_tree"+str(tree_index))

                new_leaf_indices=get_leaf_indices_recursive(tree,tree.children_left[feat_indices[order_indices][i]])
                m.addConstr(split_var[feat,feat_indices[order_indices][i],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in new_leaf_indices))

                # find parent, check if same feature, then add
                current_index=feat_indices[order_indices][i]
                parent_index= np.where(tree.children_right == current_index)[0]
                if tree.feature[parent_index]== feat:
                    m.addConstr(split_var[feat,current_index,tree_index]-split_var[feat,parent_index[0],tree_index] >= 
                                quicksum(in_leaf_var[leaf,tree_index] for leaf in new_leaf_indices))

            larger_leaf_indices=set()

            for i in range(n_ind):
                right_leaf_indices=get_leaf_indices_recursive(tree,tree.children_right[feat_indices[order_indices][n_ind-i-1]])
                m.addConstr(1 - split_var[feat,feat_indices[order_indices][n_ind-i-1],tree_index] >= quicksum(in_leaf_var[leaf,tree_index] for leaf in right_leaf_indices))

                #find parent, check if same feature, then add
                current_index=feat_indices[order_indices][n_ind-i-1]
                parent_index= np.where(tree.children_left == current_index)[0]
                if tree.feature[parent_index]== feat:
                    m.addConstr(split_var[feat,parent_index[0],tree_index]-split_var[feat,current_index,tree_index] >= 
                                quicksum(in_leaf_var[leaf,tree_index] for leaf in right_leaf_indices))

            feat_indices_all=np.append(feat_indices_all,feat_indices)
            feat_thresholds_all=np.append(feat_thresholds_all,feat_thresholds)
            tree_indices_all=np.append(tree_indices_all,np.ones(n_ind)*tree_index)

        indices_to_sort=np.argsort(feat_thresholds_all)
        ordered_thresholds=feat_thresholds_all[indices_to_sort]
        ordered_tree_indices=tree_indices_all[indices_to_sort]
        ordered_indices=feat_indices_all[indices_to_sort]

        tree.children_left

        for i in range(len(indices_to_sort)-1):
            m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       <= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])
            if ordered_thresholds[i]==ordered_thresholds[i+1]:
                m.addConstr(split_var[feat,ordered_indices[i],ordered_tree_indices[i]] 
                       >= split_var[feat,ordered_indices[i+1],ordered_tree_indices[i+1]])

    # Summing objective over all trees
    m.setObjective(quicksum(quicksum(in_leaf_var[leaf,tree_index]*tree_in_forest.tree_.value[leaf,0,0] 
                                     for leaf in np.array(get_leaf_indices(tree_in_forest.tree_))) for tree_index,tree_in_forest in enumerate(estimators))/float(num_trees),GRB.MAXIMIZE)

    m.write("new_model_file.lp")
    leaf_out=np.ones([num_trees])
    if make_relax==True:
        relax = m.relax()
        relax.optimize()
        obj = relax.getObjective()
    else:
        m.optimize()
        obj = m.getObjective()
        leaf_out=np.ones([num_trees])
        for tree_index,tree_in_forest in enumerate(estimators):
            tree=tree_in_forest.tree_
            index_leaves=np.array(get_leaf_indices(tree))
            for i in index_leaves:
                if m.getVarByName("in_leaf_var"+str(i)+"tree_index"+str(tree_index)).x==1:
                    leaf_out[tree_index]=i
    
    return obj.getValue(),leaf_out, m.Runtime


In [2]:
def make_continuous_variable_new(estimators,m,split_var,splits,num_total_features,feature_lower_bounds,feature_upper_bounds,eps):
    
    feature_var={}
    for i in range(num_total_features):
        feature_var[i]=m.addVar(lb=-GRB.INFINITY,name="feature_var"+str(i))
        
    m.update()

    for feat in range(num_total_features):
        if len(splits[feat])>0:
            const_expr_UB=[split_var[feat,split_ind]*(splits[feat][split_ind]-splits[feat][split_ind+1]) 
            for  split_ind,split_value in enumerate(splits[feat]) if split_ind < len(splits[feat])-1]+[feature_upper_bounds[feat]+ (splits[feat][len(splits[feat])-1]-feature_upper_bounds[feat])*split_var[feat,len(splits[feat])-1]]

            const_expr_LB=[split_var[feat,split_ind]*(splits[feat][split_ind-1]-splits[feat][split_ind]) 
            for  split_ind,split_value in enumerate(splits[feat]) if split_ind > 0]+[splits[feat][len(splits[feat])-1]+ (feature_lower_bounds[feat]-splits[feat][0])*split_var[feat,0]]
            m.addConstr(feature_var[feat]<= quicksum(const_expr_UB)-eps)
            m.addConstr(feature_var[feat]>= quicksum(const_expr_LB)+eps)
        else:
            m.addConstr(feature_var[feat]<= feature_upper_bounds[feat])
            m.addConstr(feature_var[feat]>= feature_lower_bounds[feat])

    m.update()

    return feature_var,m

# projected formulation
def projected_extended_random_forest_IP_unconstrained(estimators,num_total_features,feature_upper_bounds,feature_lower_bounds,x,eps,deviation_bound=False,make_relax=False):

    m = Model("RF")
    m.setParam('TimeLimit', 1800)
#     m.setParam('TimeLimit', 60)
    num_trees=len(estimators)

    print("num_total_features",num_total_features)

    #initialise variables
    in_leaf_var={}
    for tree_index,tree_in_forest in enumerate(estimators):
        tree=tree_in_forest.tree_
        index_leaves=np.array(get_leaf_indices(tree))
        for leaf in index_leaves:
            in_leaf_var[leaf,tree_index]=m.addVar(vtype=GRB.BINARY,name="in_leaf_var"+str(leaf)+"tree_index"+str(tree_index))

        m.addConstr(quicksum(in_leaf_var[leaf,tree_index] for leaf in index_leaves)==1)
    
    feature_var,m=make_continuous_variable(estimators,m,in_leaf_var,num_total_features,eps)
    m.update()
    
    if deviation_bound==True:
        m=add_deviation_bound(m,feature_var,x,num_total_features)
    
    # Summing objective over all trees
    m.setObjective(quicksum(quicksum(in_leaf_var[leaf,tree_index]*tree_in_forest.tree_.value[leaf,0,0] for leaf in np.array(get_leaf_indices(tree_in_forest.tree_))) for tree_index,tree_in_forest in enumerate(estimators))/float(num_trees),GRB.MAXIMIZE)
    m.write("extended_model_file.lp")
    
    leaf_out=np.ones([num_trees])
    feature_var_out=np.ones([num_total_features])

    if make_relax==True:
        relax = m.relax()
        relax.optimize()
        obj = relax.getObjective()
    else:
        m.optimize()

        for i in range(num_total_features):
            feature_var_out[i]=m.getVarByName("feature_var"+str(i)).x

        print("feature_var_out",feature_var_out)
        obj = m.getObjective()
        print("tight_formulation obj",obj.getValue())

        for tree_index,tree_in_forest in enumerate(estimators):
            tree=tree_in_forest.tree_
            index_leaves=np.array(get_leaf_indices(tree))
            for i in index_leaves:
                if m.getVarByName("in_leaf_var"+str(i)+"tree_index"+str(tree_index)).x==1:
                    leaf_out[tree_index]=i

    return feature_var_out,obj.getValue(),leaf_out, m.Runtime


def misic_rf_unconstrained(estimators,num_total_features,feature_upper_bounds,feature_lower_bounds,x,eps,make_continuous,deviation_bound):

    m = Model("RF")
    m.setParam('TimeLimit', 1800)
    m.setParam('TimeLimit', 60)
    num_trees=len(estimators)

    print("num_total_features",num_total_features)

    #initialise variables
    in_leaf_var={}
    split_var={}
    tree_obj={}

    # get 2D list of splits
    splits=get_ordered_thresholds_for_each_feature(estimators, num_total_features)

    for feat in range(num_total_features):
        for split_ind,split_value in enumerate(splits[feat]):
            split_var[feat,split_ind]=m.addVar(vtype=GRB.BINARY,name="split_var"+str(feat)+"-"+str(split_ind))

    for tree_index,tree_in_forest in enumerate(estimators):

        tree=tree_in_forest.tree_
        num_nodes=tree.node_count
        index_leaves=np.array(get_leaf_indices(tree))
        ## interior nodes
        index_splits=np.where(tree.children_left!= (-1))[0]

        for i in index_leaves:
            in_leaf_var[i,tree_index]=m.addVar(name="in_leaf_var"+str(i)+"tree_index"+str(tree_index))

        tree_obj[tree_index]=m.addVar(name='tree_obj'+str(tree_index))

        m.update()
        m.addConstr(tree_obj[tree_index]==quicksum(in_leaf_var[leaf,tree_index]*tree.value[leaf,0,0] for leaf in index_leaves))
        binary_path=get_binary_path_to_leaf(np.ones(num_nodes)*(-1),tree,0)

        # branching constraint
        for split in index_splits:
            left_descendents=[]
            right_descendents=[]

            if tree.children_left[split] in index_leaves:
                left_descendents.append(tree.children_left[split])

            if tree.children_right[split] in index_leaves:
                right_descendents.append(tree.children_right[split])

            for leaf in index_leaves:
                # check if this leaf is a descendent of this split
                if not(binary_path[leaf][tree.children_left[split]] == -1):
                    left_descendents.append(leaf)
                if not(binary_path[leaf][tree.children_right[split]] == -1):
                    right_descendents.append(leaf)

            #find corresp thresh
            split_pos=splits[tree.feature[split]].index(tree.threshold[split])

            m.addConstr(quicksum(in_leaf_var[desc,tree_index] for desc in left_descendents)<= split_var[tree.feature[split],split_pos])
            m.addConstr(quicksum(in_leaf_var[desc,tree_index] for desc in right_descendents)<= 1-split_var[tree.feature[split],split_pos])

        m.addConstr(quicksum(in_leaf_var[leaf,tree_index] for leaf in index_leaves)==1)

    for feat in range(num_total_features):
        for split_ind,split_value in enumerate(splits[feat]):
            if split_ind<len(splits[feat])-1:
                m.addConstr(split_var[feat,split_ind] <=split_var[feat,split_ind+1])
                
    if make_continuous==True: 
        feature_var,m=make_continuous_variable_new(estimators,m,split_var,splits,num_total_features,feature_lower_bounds,feature_upper_bounds,eps)
        m.update()
        if deviation_bound==True:
            m=add_deviation_bound(m,feature_var,x,num_total_features)
            
        
    m.update()   

    # Summing objective over all trees
    m.setObjective(quicksum(tree_obj[tree_index] for tree_index,tree_in_forest in enumerate(estimators))/float(num_trees),GRB.MAXIMIZE)
    m.write("unconst_model_file.lp")
    m.optimize()
    obj = m.getObjective()

    if make_continuous==True: 
        print("in loop")
        feature_var_out=np.ones([num_total_features])
        for i in range(num_total_features):
            feature_var_out[i]=m.getVarByName("feature_var"+str(i)).x
    else:
        feature_var_out=np.ones([num_total_features])
        for feat in range(num_total_features):
            if len(splits[feat])>0:
                feature_var_out[feat]=max(splits[feat])+eps
                chosen_split=max(splits[feat])
                for split_ind,split_value in enumerate(splits[feat]):
                    var_sol=m.getVarByName("split_var"+str(feat)+"-"+str(split_ind)).x
                    if var_sol==1:
                        if split_value < chosen_split:
                            feature_var_out[feat]=split_value-eps
                            chosen_split=split_value

                    
    leaf_out=np.ones([num_trees])
    for tree_index,tree_in_forest in enumerate(estimators):
        tree=tree_in_forest.tree_
        index_leaves=np.array(get_leaf_indices(tree))
        for i in index_leaves:
            if m.getVarByName("in_leaf_var"+str(i)+"tree_index"+str(tree_index)).x==1:
                leaf_out[tree_index]=i
                

    print('feature_var_out',feature_var_out)
    print('obj.getValue()',obj.getValue())
    return feature_var_out, obj.getValue(),leaf_out


def make_continuous_variable(estimators,m,in_leaf_var,num_total_features,eps):
    
    feature_var={}
    #The variable x_i in the model
    for i in range(num_total_features):
        feature_var[i]=m.addVar(lb=-GRB.INFINITY,name="feature_var"+str(i))

    for tree_index,tree_in_forest in enumerate(estimators):

        tree=tree_in_forest.tree_
        num_nodes=tree.node_count
        ## leaf nodes
        index_leaves=np.array(get_leaf_indices(tree))
        n_leaves=index_leaves.size

        # 0 for left, 1 for right to all leaves in a dictionary
        binary_path=get_binary_path_to_leaf(np.ones(num_nodes)*(-1),tree,0)

        m.update()

        # demand constraint
        upper_bound_leaf=np.ones([num_nodes,num_total_features])
        lower_bound_leaf=np.ones([num_nodes,num_total_features])

        for leaf in index_leaves:

            index_right=np.where(binary_path[leaf]==1)[0]
            index_left=np.where(binary_path[leaf]==0)[0]
            upper_threshold=tree.threshold[index_left]
            lower_threshold=tree.threshold[index_right]
            feature_split_left=tree.feature[index_left]
            feature_split_right=tree.feature[index_right]

            for feat in range(num_total_features):
                if feat in feature_split_left:
                    split_loc=np.where(feature_split_left==feat)[0]
                    upper_bound_leaf[leaf,feat]=min(upper_threshold[split_loc])
                else: upper_bound_leaf[leaf,feat] = feature_upper_bounds[feat]

                if feat in feature_split_right:
                    split_loc=np.where(feature_split_right==feat)[0]
                    lower_bound_leaf[leaf,feat]=max(lower_threshold[split_loc])
                else: lower_bound_leaf[leaf,feat] = feature_lower_bounds[feat]


        for feat in range(num_total_features):
            m.addConstr(feature_var[feat]>=eps+quicksum(in_leaf_var[leaf,tree_index]*lower_bound_leaf[leaf,feat] for leaf in index_leaves))
            m.addConstr(feature_var[feat]+eps<=quicksum(in_leaf_var[leaf,tree_index]*upper_bound_leaf[leaf,feat] for leaf in index_leaves))
    m.update()
    return feature_var,m




In [29]:


def line(error_y_mode=None, **kwargs):
    """Extension of `plotly.express.line` to use error bands."""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.1)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        # Reorder data as said here: https://stackoverflow.com/a/66854398/8849755
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)

    return fig